In [387]:
import voxel_slam

In [388]:
import numpy as np
import mrob 
import open3d as o3d 

import pathlib
import copy
import distinctipy

### I/O 

In [389]:
def read_depth_to_rgb_map(association_path):
    depth_to_rgb_map = dict()
    with open(association_path) as data:
        for line in data:
            _, color_name, _, depth_name = line.split()
            color_name = color_name.strip('\n').lstrip("color/")
            depth_name = depth_name.strip('\n').lstrip("depth/")
            depth_to_rgb_map[depth_name] = color_name
    return depth_to_rgb_map

def parse_trajectories(input_path):
    ts = dict()
    with open(input_path) as data:
        for line in data:
            if line.startswith('#'):
                continue
            line_tokens = line.strip('\n').split()
            
            timestamp = line_tokens[0].replace('.', '')
            timestamp = "0" * (12 - len(timestamp)) + timestamp
            
            trajectory = np.asarray(list(map(float, line_tokens[1:])))
            ts.update({timestamp: trajectory})
    return ts


def filename_to_pose(depth_filename: str, depth_to_rgb_map: str, poses: dict):
    rgb_filename = depth_to_rgb_map[depth_filename].rstrip(".png")
    return poses["0" * (12 - len(rgb_filename)) + rgb_filename]


def trajectory_to_se3(trajectory):
    t, Q = trajectory[:3], trajectory[3:]
    R = mrob.geometry.SO3(mrob.geometry.quat_to_so3(Q))
    return mrob.geometry.SE3(R, t)

def parse_trajectories_float(input_path, ts_multiplier=1e9):
    ts = dict()
    with open(input_path) as data:
        for line in data:
            if line.startswith('#'):
                continue
            line_tokens = line.strip('\n').split()
            
            timestamp = float(line_tokens[0]) * ts_multiplier
            
            trajectory = np.asarray(list(map(float, line_tokens[1:])))
            ts.update({timestamp: trajectory})
    return ts

In [390]:
def closest_argmin(A, B):
    L = B.size
    sidx_B = B.argsort()
    sorted_B = B[sidx_B]
    sorted_idx = np.searchsorted(sorted_B, A)
    sorted_idx[sorted_idx==L] = L-1
    mask = (sorted_idx > 0) & \
    ((np.abs(A - sorted_B[sorted_idx-1]) < np.abs(A - sorted_B[sorted_idx])) )
    return sidx_B[sorted_idx-mask]

In [391]:
def break_on_sectors(pcd, number_of_sectors):
    sector_rad = 2 * np.pi / number_of_sectors
    sectors = {}
    for i in range(number_of_sectors):
        sectors[i * sector_rad, (i + 1) * sector_rad] = []

    for point_i, point in enumerate(np.asarray(pcd.points)):
        angle = np.arctan2(point[1], point[0])
        if angle < 0:
            angle += 2 * np.pi
        
        for angle_range in sectors:
            if angle_range[0] <= angle <= angle_range[1]:
                sectors[angle_range].append(point_i)
                break    

    return sectors

In [397]:
def read_hilti_sequence(ts_to_quat, 
                        ts_to_depth_path,
                        ts_to_imu, 
                        pose_to_imu_assoc,
                        number_of_sectors=30,
                        start_of_sequence=0, 
                        number_of_clouds=-1, 
                        center_distance_threshold=1.5):
    poses = []
    clouds = []
    undistorted_clouds = []

    lidar_so3 = mrob.geometry.SO3(mrob.geometry.quat_to_so3(np.asarray([ 0.7071068, -0.7071068, 0, 0 ])))
    lidar_t = np.asarray([ -0.001, -0.00855, 0.055 ])   
    imu_to_lidar_se3 = mrob.geometry.SE3(lidar_so3, lidar_t).T()

    max_angular_norm = None
    max_anglular_norm_id = -1

    for i, ts in enumerate(sorted(ts_to_quat)[start_of_sequence : start_of_sequence + number_of_clouds]):
        imu_pose = trajectory_to_se3(ts_to_quat[ts]).T()

        pose = imu_pose @ imu_to_lidar_se3
        cloud = o3d.io.read_point_cloud(str(ts_to_depth_path[ts]))
        cloud_points = np.asarray(cloud.points)
        f = np.where(np.linalg.norm(cloud_points, axis=1) > center_distance_threshold)[0] 
        cloud.points = o3d.utility.Vector3dVector(cloud_points[f])
        cloud.paint_uniform_color([0.0, 0.0, 0.0])
        clouds.append(cloud)

        # Ego motion undistortion
        number_of_sectors = 30
        scan_time = 0.1
        sectors = break_on_sectors(cloud, number_of_sectors=number_of_sectors)

        cloud_imu = ts_to_imu[pose_to_imu_assoc[ts]]
        angular_velocity = cloud_imu[-3:]

        if i == 0:
            angular_velocity = np.zeros(3)

        if max_angular_norm is None:
            max_angular_norm = np.linalg.norm(angular_velocity)
            max_anglular_norm_id = i

        if np.linalg.norm(angular_velocity) > max_angular_norm:
            max_angular_norm = np.linalg.norm(angular_velocity)
            max_anglular_norm_id = i

        undistorted_cloud = o3d.geometry.PointCloud()

        for i, sector_point_idx in enumerate(sectors.values()):
            if len(sector_point_idx) == 0:
                continue
            # TODO: Identity
            sector_multiplier = (i + 1) * (scan_time / number_of_sectors)
            sector_velocity = sector_multiplier * angular_velocity
            imu_sector_transform = mrob.SE3(mrob.SO3(sector_velocity), np.zeros(3))
            
            dT = np.linalg.inv(imu_to_lidar_se3) @ np.linalg.inv(imu_sector_transform.T()) @ imu_to_lidar_se3
            sector_cloud = cloud.select_by_index(sector_point_idx)
            sector_cloud.transform(dT)
            undistorted_cloud += sector_cloud


        # #######################
        
        poses.append(pose)
        undistorted_clouds.append(undistorted_cloud)

    print(max_angular_norm, max_anglular_norm_id)
    return clouds, undistorted_clouds, poses

In [398]:
clouds_path = "/home/ach/Desktop/datasets/hilti/out2"
poses_path = "/home/ach/Desktop/datasets/hilti/exp14_basement_2_imu.txt"
imu_path = "/home/ach/Desktop/datasets/hilti/hilti_imu.txt"

In [399]:
ts_multiplier = 1 / 1e9
ts_to_depth_path = {float(x.stem) * ts_multiplier : x for x in pathlib.Path(clouds_path).iterdir()}
ts_to_quat = parse_trajectories_float(poses_path, ts_multiplier)
ts_to_imu = parse_trajectories_float(imu_path, ts_multiplier=ts_multiplier)

In [400]:
imu_ts = np.asarray(list(ts_to_imu.keys()), dtype=np.float64) 
poses_ts = np.asarray(list(ts_to_quat.keys()), dtype=np.float64)
closest = closest_argmin(poses_ts, imu_ts)
pose_imu_assoc = {poses_ts[i]: imu_ts[closest[i]] for i in range(len(poses_ts))} 

In [401]:
clouds, undistorted_clouds, poses = read_hilti_sequence(ts_to_quat, 
                                    ts_to_depth_path, 
                                    ts_to_imu,
                                    pose_to_imu_assoc=pose_imu_assoc,
                                    number_of_sectors=10, 
                                    start_of_sequence=310, 
                                    number_of_clouds=5, 
                                    center_distance_threshold=3)

0.4670675374487492 4


In [384]:
for pcd in clouds:
    pcd.paint_uniform_color([1.0, 0.0 , 0.0])

for pcd in undistorted_clouds:
    pcd.paint_uniform_color([0.0, 1.0 , 0.0])

In [385]:
for i in range(len(clouds)):
    o3d.visualization.draw_geometries([clouds[i], undistorted_clouds[i]])

In [371]:
o3d.visualization.draw_geometries([voxel_slam.aggregate_map(undistorted_clouds, poses)])

In [373]:
o3d.visualization.draw_geometries([voxel_slam.aggregate_map(clouds, poses), voxel_slam.aggregate_map(undistorted_clouds, poses)])